## Algorithm 1


In [1]:
import contextlib
import functools
import time
import random

import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

from sklearn.metrics import mean_squared_error
from abc import ABC, abstractmethod


def get_matrices(weight_vec, B):
    Sigma = np.diag(np.full(weight_vec.shape, 0.9 / 2))

    D = B

    Gamma_vec = np.array((1.0 / (np.sum(abs(B), 0)))).ravel()
    Gamma = np.diag(Gamma_vec)

    if np.linalg.norm(np.dot(Sigma ** 0.5, D).dot(Gamma ** 0.5), 2) > 1:
        print ('product norm', np.linalg.norm(np.dot(Sigma ** 0.5, D).dot(Gamma ** 0.5), 2))
        # raise Exception('higher than 1')
    return Sigma, Gamma, Gamma_vec, D


def algorithm_1(K, B, weight_vec, X, Y, samplingset, lambda_lasso, score_func=mean_squared_error, loss_func='linear_reg'):
    # calculate the needed matrices for the algorithm 1 from adjacency matrix and weight vector of the empirical graph G
    Sigma, Gamma, Gamma_vec, D = get_matrices(weight_vec, B)

    E, N = B.shape
    m, n = X[0].shape

    # initialize the loss funcion
    if loss_func == 'linear_reg':
        optimizer = LinearOptimizer(samplingset, Gamma_vec, X, Y)
    elif loss_func == 'logistic_reg':
        optimizer = LogisticOptimizer(Gamma_vec, X, Y)
    else:
        print('invalid loss_func')
        return

    new_w = np.array([np.zeros(n) for i in range(N)])
    new_u = np.array([np.zeros(n) for i in range(E)])

    iteration_scores = []
    limit = np.array([np.zeros(n) for i in range(E)])
    for i in range(n):
        limit[:, i] = lambda_lasso * weight_vec

    for iterk in range(K):
        if iterk % 100 == 0:
            print ('iter:', iterk)
            
        prev_w = np.copy(new_w)

        hat_w = new_w - np.dot(Gamma, np.dot(D.T, new_u))

        for i in range(N):
            if i in samplingset:
                # line 3 of algorithm 1
                new_w[i] = optimizer.optimize(i, hat_w)
            else:
                # line 6 of algorithm 1
                new_w[i] = hat_w[i]

        # line 8 of algorithm 1
        new_u = new_u + np.dot(Sigma, np.dot(D, 2 * new_w - prev_w))

        # line 9 of algorithm 1
        normalized_u = np.where(abs(new_u) >= limit)
        new_u[normalized_u] = limit[normalized_u] * new_u[normalized_u] / abs(new_u[normalized_u])

        Y_pred = []
        for i in range(N):
            Y_pred.append(np.dot(X[i], new_w[i]))

        iteration_scores.append(score_func(Y.reshape(N, m), Y_pred))

    # print (np.max(abs(new_w - prev_w)))

    return iteration_scores, new_w


## Deep Learning Experiment


In [2]:
Image_Width = 150
Image_Height = 150
Image_Size = (Image_Width, Image_Height)
Image_Channels = 3

BATCH_SIZE = 32
EPOCHS = 3

## Neural Network Models

In [3]:
import keras
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf

from keras import layers


# get the base(pre-trained) model data
def get_base_model_data():
    data_augmentation = keras.Sequential(
        [
            layers.experimental.preprocessing.RandomFlip("horizontal"),
            layers.experimental.preprocessing.RandomRotation(0.1),
        ]
    )

    base_model = keras.applications.Xception(
        weights="imagenet",  # Load weights pre-trained on ImageNet.
        input_shape=(Image_Width, Image_Height, Image_Channels),
        include_top=False,
    )  # Do not include the ImageNet classifier at the top.

    # Freeze the base_model
    base_model.trainable = False

    # Create new model on top
    inputs = keras.Input(shape=(Image_Width, Image_Height, Image_Channels))
    x = data_augmentation(inputs)  # Apply random data augmentation

    # Pre-trained Xception weights requires that input be normalized
    # from (0, 255) to a range (-1., +1.), the normalization layer
    # does the following, outputs = (inputs - mean) / sqrt(var)
    norm_layer = keras.layers.experimental.preprocessing.Normalization()
    mean = np.array([127.5] * 3)
    var = mean ** 2
    # Scale inputs to [-1, +1]
    x = norm_layer(x)
    norm_layer.set_weights([mean, var])

    # The base model contains batchnorm layers. We want to keep them in inference mode
    # when we unfreeze the base model for fine-tuning, so we make sure that the
    # base_model is running in inference mode here.
    x = base_model(x, training=False)
    return x, inputs


# get the base(pre-trained) model
def get_base_model():
    x, inputs = get_base_model_data()
    outputs = keras.layers.GlobalAveragePooling2D()(x)

    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=[keras.metrics.BinaryAccuracy()],
    )

    return model


# get the new(trainable) model
def get_new_model():
    inputs = keras.Input(shape=(2048,))

    x = keras.layers.Dropout(0.2)(inputs)
    outputs = keras.layers.Dense(1)(x)

    extra_model = keras.Model(inputs, outputs)

    extra_model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=[keras.metrics.BinaryAccuracy()],
    )

    return extra_model


# get the model
def get_NN_model():
    # base model
    x, inputs = get_base_model_data()

    # new model
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
    outputs = keras.layers.Dense(1)(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=[keras.metrics.BinaryAccuracy()],
    )

    for layer in model.layers[:-1]:
        layer.trainable = False

    return model


# calculate base model output and true labels for all images
def get_base_model_output():

    # get the base model
    base_model = get_base_model()

    # load the data from tensorflow dataset (all the images)
    (dataset,), metadata = tfds.load(
        "cats_vs_dogs",
        split=["train[:100%]"],
        shuffle_files=True,
        with_info=True,
    )

    # resize the images of the dataset to the standard size
    dataset = dataset.map(lambda item: (tf.image.resize(item['image'], Image_Size), item['label']))
    dataset = dataset.cache().batch(BATCH_SIZE).prefetch(buffer_size=10)

    # get the output of the base model for the dataset
    base_model_outputs = base_model.predict(dataset)
    '''
    base_model_output: A list containing the output of the base(pre-trained) model for all the images
    '''

    # obtain the true labels for the dataset
    true_labels = []
    for obj in dataset:
        true_labels += list(np.array(obj[1]))
    true_labels = np.array(true_labels)
    '''
    true_labels: A list containing the true label of all the images (which is 0 or 1 for each image)
    '''

    return base_model_outputs, true_labels


## Load Saved(Trained) Data


In [4]:
import json
import os

import numpy as np


# Return the output of the base model and the train vector for the training dataset of this model
def get_model_train_images_data(train_data_image_names, all_images_indices, all_images_size, base_model_outputs, true_labels):
    train_images_vector = np.zeros(all_images_size)
    base_model_output = []
    model_labels = []
    for train_image_name in train_data_image_names:
        index = all_images_indices[train_image_name]
        train_images_vector[index] = 1
        model_labels.append(true_labels[index])
        item_predict = np.concatenate((base_model_outputs[index], [1]))  # [1] is for the bias (b)
        base_model_output.append(item_predict)
    return base_model_output, train_images_vector, model_labels


def get_all_images_dict(data):
    all_images_indices = {}
    cnt = 0
    for train_data in data:
        for i, file_name in enumerate(train_data['train_df']):
            if file_name not in all_images_indices:
                all_images_indices[file_name] = cnt
                cnt += 1
    return all_images_indices


def get_trained_model_weights(raw_model_weights):
    model_weights = []
    for weight in raw_model_weights[-2:]:
        model_weights.append(np.array(weight))
    model_weights = np.array(model_weights)
    return model_weights


# get the trained dataset and weights of each trained model and also the features of algorithm 1
def parse_saved_data(data, base_model_output, true_labels):

    all_images_indices = get_all_images_dict(data)
    '''
    all_images_indices: a dictionary from image_name to index
    '''
    all_images_size = len(all_images_indices.keys())
    '''
    all_images_size : total number of images of the (tensorflow) dataset
    '''

    all_models_train_images = []
    all_models_weights = []
    X = []
    '''
    X: A list containing the output of the base model for trainset of each model, which is the features of algorithm 1
    '''
    Y = []
    '''
    Y: A list containing the true labels for trainset of each model, which is the labels of algorithm 1
    '''
    for model_data in data:

        base_model_train_images_output, model_train_images, model_labels = get_model_train_images_data(model_data['train_df'], all_images_indices, all_images_size, base_model_output, true_labels)
        '''
        base_model_train_images_output: the output of the base model for the training dataset of this model
        model_train_images: a vector from 0/1 with the size of "all_images_size", model_train_images[i] = 1 if 
                the image with the index i is in the train dataset of this model otherwise model_train_images[i] = 0
        model_labels: the true labels for the training dataset of this model
        '''

        X.append(np.array(base_model_train_images_output))
        Y.append(np.array(model_labels))

        all_models_train_images.append(model_train_images)

        model_weights = get_trained_model_weights(model_data['weights'])
        '''
        model_weights: the weights of this model for the new model (trainable layers)
        '''
        all_models_weights.append(model_weights)

    X = np.array(X)
    Y = np.array(Y)

    return all_models_train_images, all_models_weights, X, Y


# read the trained models data from saved files
def read_trained_data_from_saved_files(train_data_dir):
    data = []
    for filename in sorted(os.listdir(train_data_dir)):
        if '.json' not in filename:
            continue
        num = filename.split('_')[-1].replace('.json', '')
        if int(num) >= 50:
            continue
        with open(os.path.join(train_data_dir, filename), 'r') as f:
            data.append(json.load(f))
    return data


def load_trained_data(train_data_dir, base_model_output, true_labels):

    # read the trained models data from saved files
    data = read_trained_data_from_saved_files(train_data_dir)
    '''
    data: saved data from the trained models
    '''

    # get the trained dataset and weights of each trained model and also the features of algorithm 1
    all_models_train_images, all_models_weights, X, Y = parse_saved_data(data, base_model_output, true_labels)
    '''
    all_models_train_images: A list containing the images used for training each model
    all_models_weights : A list containing the weight of the new model based on training each model
    X : A list containing the output of the base model for trainset of each model, which is the features of algorithm 1
    Y : A list containing the true labels for trainset of each model, which is the labels of algorithm 1
    '''

    return all_models_train_images, all_models_weights, X, Y

## Create graph


In [5]:
# calculate the distance between the given nodes of the graph
def get_dist(first_node, second_node):
    all_equals = np.where(first_node == second_node)[0]
    equal_train_images = np.where(first_node[all_equals] == 1)[0]
    dist = len(equal_train_images) / len(np.where(first_node == 1)[0])
    return dist


# calculate the adjacency matrix and the weight vector of the empirical graph G
def get_B_and_weight_vec(all_models_train_images, neigh_cnt=3):
    '''
    
    :param trained_models_train_images: A list containing the images used for training each model
    :param neigh_cnt: number of the neighbors for each node of the empirical graph G

    '''
    
    N = len(all_models_train_images)
    E = int(N * (N - 1) / 2)

    weight_vec = np.zeros(E)
    '''
    the weight vector of the edges of the empirical graph G
    '''
    B = np.zeros((E, N))
    '''
    the adjacency matrix of the empirical graph G
    '''
    
    cnt = 0
    '''
    number of edges of the empirical graph G
    '''
    for i in range(N):
        node_dists = []
        '''
        a list containing the distance between node i and other nodes of the graph
        '''
        for j in range(N):
            if j == i:
                continue
            node_dists.append(get_dist(all_models_train_images[i], all_models_train_images[j]))
        
        # sort node_dists in order to pick the nearest nodes to the node i 
        node_dists.sort(reverse=True)

        node_cnt = 0
        for j in range(N):
            
            if node_cnt >= neigh_cnt:
                break
                
            if j == i:
                continue
                
            # calculate the distance between node i and j of the graph
            dist = get_dist(all_models_train_images[i], all_models_train_images[j])
            if dist == 0 or dist < node_dists[neigh_cnt]:
                continue

            node_cnt += 1
            B[cnt][i] = 1
            B[cnt][j] = -1
            weight_vec[cnt] = dist
            cnt += 1

    B = B[:cnt, :]
    weight_vec = weight_vec[:cnt]
    return B, weight_vec



## Save figures


In [6]:
import numpy as np
import matplotlib.pyplot as plt


# calculate new model output for all images
def get_new_model_output(new_model_weights, base_model_output):
    new_model_weights = [np.array(new_model_weights[:-1]).reshape(-1, 1), np.array(new_model_weights[-1:])]

    new_model = get_new_model()
    new_model.set_weights(new_model_weights)

    model_predicts = new_model.predict(base_model_output).flatten()
    model_predicts[model_predicts > 0] = 1
    model_predicts[model_predicts <= 0] = 0

    return model_predicts


def save_figures(alg1_estimated_weights, original_weights, lambda_lasso, base_model_output, true_labels):
    '''

    :param alg1_estimated_weights: A list containing the weights of the models estimated by algorithm 1
    :param original_weights: A list containing the weights of the models based on training each model
    :param lambda_lasso: lambda_lasso parameter used for algorithm 1
    :param base_model_output: A list containing the output of the base model(pre-trained model) for all the images
    :param true_labels: A list containing the true label of all the images

    '''

    N = len(alg1_estimated_weights)

    alq1_scores = []  # blue curve
    trained_model_scores = []  # orange curve

    for i in range(N):

        # the trained model output for all images
        trained_model_output = get_new_model_output(original_weights[i], base_model_output)
        # orange curve
        trained_model_score = np.where(true_labels == trained_model_output)[0].shape[0] / len(true_labels)
        trained_model_scores.append(trained_model_score)

        # alg1 output for all images
        alg1_output = get_new_model_output(alg1_estimated_weights[i], base_model_output)
        # blue curve
        alg1_score = np.where(true_labels == alg1_output)[0].shape[0] / len(alg1_output)
        alq1_scores.append(alg1_score)

    x_axis = [i for i in range(N)]
    plt.close()
    plt.plot(x_axis, alq1_scores, label='our')
    plt.plot(x_axis, trained_model_scores, label='deep learning')
    plt.title('alg1 vs trained accuracy')
    plt.xlabel('model')
    plt.ylabel('accuracy')
    plt.legend(loc="lower left")
    plt.savefig('deep_learning_lasso/train_accuracy_%s.png' % lambda_lasso)


## Optimizers

In [7]:
class Optimizer(ABC):

    @abstractmethod
    def optimize(self, idx, hat_w):
        pass



### Linear Optimizer

In [8]:
class LinearOptimizer(Optimizer):

    def __init__(self, samplingset, Gamma_vec, X, Y):
        super(Optimizer).__init__()
        self.MTX1_INV, self.MTX2 = self.get_preprocessed_matrices(samplingset, Gamma_vec, X, Y)

    def get_preprocessed_matrices(self, samplingset, Gamma_vec, X, Y):
        MTX1_INV = {}
        MTX2 = {}
        for i in samplingset:
            mtx1 = 2 * Gamma_vec[i] * np.dot(X[i].T, X[i]).astype('float64')
            if mtx1.shape:
                mtx1 += 1 * np.eye(mtx1.shape[0])
                mtx_inv = np.linalg.inv(mtx1)
            else:
                mtx1 += 1
                mtx_inv = 1.0 / mtx1
            MTX1_INV[i] = mtx_inv

            MTX2[i] = 2 * Gamma_vec[i] * np.dot(X[i].T, Y[i]).T
        return MTX1_INV, MTX2


    def optimize(self, idx, hat_w):
        mtx2 = hat_w[idx] + self.MTX2[idx]
        mtx_inv = self.MTX1_INV[idx]

        return np.dot(mtx_inv, mtx2)

### Logistic Optimizer

In [9]:
class LogisticOptimizer(Optimizer):

    def __init__(self, tau, X, Y):
        super(Optimizer).__init__()
        self.tau = tau
        self.X = tf.constant(X, dtype=tf.float64)
        self.Y = tf.constant(Y, dtype=tf.float64)

    def optimize(self, idx, hat_w):
        def make_val_and_grad_fn(value_fn):
            @functools.wraps(value_fn)
            def val_and_grad(x):
                return tfp.math.value_and_gradient(value_fn, x)

            return val_and_grad

        @contextlib.contextmanager
        def timed_execution():
            t0 = time.time()
            yield
            dt = time.time() - t0
            print('Evaluation took: %f seconds' % dt)

        def np_value(tensor):
            if isinstance(tensor, tuple):
                return type(tensor)(*(np_value(t) for t in tensor))
            else:
                return tensor.numpy()

        def run(optimizer):
            optimizer()
            # with timed_execution():
            result = optimizer()
            return np_value(result)

        def regression_loss(params):
            
            # Calculate the logistic loss
            labels = Y[idx]
            feature = X[idx]
            
            reshaped_params = tf.expand_dims(params, 1)
            logits = tf.matmul(feature, reshaped_params)
            labels = tf.expand_dims(labels, 1)
            mse_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits))

            # calculate the penalty loss
            w = tf.expand_dims(tf.constant(hat_w[idx], dtype=tf.float64), 1)
            penalty_var = tf.math.subtract(w, params)
            loss_penalty = regularization_factor * tf.nn.l2_loss(penalty_var)

            
            total_loss = mse_loss + loss_penalty

            return total_loss

        @tf.function
        def l1_regression_with_lbfgs():
            return tfp.optimizer.lbfgs_minimize(
                make_val_and_grad_fn(regression_loss),
                initial_position=tf.constant(start),
                tolerance=1e-8)

        dim = len(hat_w[idx])
        start = np.random.randn(dim)
        X = self.X
        Y = self.Y

        regularization_factor = 1/(2*self.tau[idx])

        results = run(l1_regression_with_lbfgs)
        minimum = results.position
        return minimum




## Main


In [10]:
def get_Y_and_W(X, trained_models_weights):
    '''
    :param X: A list of the features of algorithm 1
    :param trained_models_weights: A list containing the weight of the new model based on training each model
    '''

    Y = []
    W = []
    for i in range(len(X)):

        # The weights of the trainable layers (the new model) of the i_th trained model
        weights = trained_models_weights[i]

        # the weights of the dense layer of the model
        w1 = np.array(weights[-2]).flatten()

        # the bias of the dense layer
        w2 = weights[-1]

        # combining the weights and the bias of the dense layer of the model, which is the weight of the node for alg1
        w = np.concatenate((w1, w2))
        W.append(w)

        # the label of the i_th node for alg1
        Y.append(X[i].dot(w))

    Y = np.array(Y)
    W = np.array(W)

    return Y, W


def deep_learning_run(lambda_lasso, K=1000, train_data_dir='deep_learning_lasso/new_deeplarning_data'):

    # calculate base model output and true labels for all images
    base_model_output, true_labels = get_base_model_output()
    '''
    base_model_output: A list containing the output of the base(pre-trained) model for all the images
    true_labels: A list containing the true label of all the images (which is 0 or 1 for each image)
    '''

    # load trained data from saved models in train_data_dir
    trained_models_train_images, trained_models_weights, X, Y = load_trained_data(train_data_dir, base_model_output, true_labels)
    '''
    trained_models_train_images: A list containing the images used for training each model
    trained_models_weights : A list containing the weight of the new model based on training each model
    X : A list containing the output of the base model for trainset of each model, which is the features of algorithm 1
    Y : A list containing the true labels for trainset of each model, which is the labels of algorithm 1
    '''

    # create B and weight_vec for the empirical graph G
    B, weight_vec = get_B_and_weight_vec(trained_models_train_images)
    E, N = B.shape
    '''
    B : Incidence matrix of the empirical graph G
    weight_vec : Wight of each edge of the empirical graph G
    '''

    # calculate the weights(W) of the empirical graph G
    _, W = get_Y_and_W(X, trained_models_weights)
    print ("hereee", Y.shape, W.shape, X.shape, true_labels.shape)
    '''
    W : The weights of the nodes for the algorihtm 1
    '''
    
    # choose sampling set for alg1
    M=0.2
    samplingset = random.sample([i for i in range(N)], k=int(M * N))
    # samplingset = [53, 92, 99, 19, 16, 32, 6, 9, 39, 43, 34, 54, 23, 8, 13, 88, 1, 62, 22, 60]
    '''
    samplingset : The samplingset selected for algorithm 1
    '''

    print ('start alg')
    # alg1
    K=2
    _, alg1_estimated_weights = algorithm_1(K, B, weight_vec, X, Y, samplingset, lambda_lasso, loss_func='logistic_reg')    
    '''
    alg1_estimated_weights : The estimated weights by algorithm 1
    '''

    # save the orange and blue fig
    save_figures(alg1_estimated_weights, W, lambda_lasso, base_model_output, true_labels)

    return alg1_estimated_weights


In [ ]:
res = deep_learning_run(0.001)